In [2]:
import tensorflow as tf

import pandas as pd

import numpy as np

2024-01-21 08:47:30.913365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 08:47:30.913393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 08:47:30.914618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-21 08:47:30.921821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 08:47:31.761657: W tensorflow/compiler/tf2

In [11]:
max_features = 2800
max_len = 20


X_train = pd.read_csv('train.csv')
X_train = X_train[['Surname', 'Gender', 'Geography']]
print(len(X_train), 'Training sequences')

115524 Training sequences


In [12]:
col_dict = dict.fromkeys(X_train.columns, '')

X_train.rename(columns = col_dict, inplace=True)

X_train

,,,
0,Okwudiliolisa,Male,France
1,Hsueh,Male,France
2,Kao,Male,France
3,Chiemenam,Male,Spain
4,Genovese,Male,Germany
...,...,...,...
115519,T'ang,Female,France
115520,Ifeatu,Male,Germany
115521,Oluchukwu,Male,France
115522,Okechukwu,Male,France


In [13]:
X_train = tf.keras.utils.pad_sequences(X_train, maxlen=max_len)


In [14]:
inputs = tf.keras.layers.Input(shape=(None,), dtype='int32')

x = tf.keras.layers.Embedding(max_features, 128)(inputs)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

2024-01-21 08:54:29.862785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-21 08:54:29.881229: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-21 08:54:29.881419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         358400    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 556161 (2.12 MB)
Trainable params: 556161 (2.12

In [15]:
model.compile(optimizer='Adam',
             loss='binary_crossentropy',
             metrics=['acc'])

In [17]:
model.fit(X_train, batch_size=32)

ValueError: in user code:

    File "/home/mllabs/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/mllabs/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/mllabs/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/mllabs/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1152, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/mllabs/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1106, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=binary_crossentropy, and therefore expects target data to be